# Aula 4 - Busca competitiva

Neste notebook, implementaremos um jogo simples e o algoritmo Alpha-Beta Pruning para resolver o mesmo. O objetivo aqui é entender como este algoritmo funciona. Ao final, você deverá realizar os exercícios propostos.

 Para começar, seguem abaixo algumas definições iniciais.

In [ ]:
import numpy as np
import random
from collections import namedtuple
from google.colab import output
import time

Inicialmente, criaremos uma classe genérica para jogos com `n` jogadores com movimentos alternados. Note que esta classe é parecida com a definição de problemas que vimos antes. A diferença é que agora precisamos definir uma utilidade para cada estado e também um teste de estado final; anteriormente, tínhamos o custo de um caminho e um teste de objetivo. Para criar um novo jogo, basta criar uma subclasse de `Game` e implementar os métodos `actions`, `result`, `utility` e `terminal_test`. As funções `display` e `to_move` podem ser sobrescritas, se necessário, por suas correspondentes específicas ao problema. Ao criar a subclasse, é necessário ainda definir o estado inicial `self.initial` no construtor da classe.

In [ ]:
class Game:
    
    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)
        
    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players, animate=True):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if animate:
                  self.display(state)
                if self.terminal_test(state):
                    if not animate:
                      self.display(state)
                    return self.utility(state, self.to_move(self.initial))

Agora, vamos criar uma classe para o jogo da velha (Tic-Tac-Toe). O construtor recebe três argumentos. Os argumentos `h` e `v` denotam a largura e altura do tabuleiro. O argumento `k` denota o número de peças que devem ser alinhadas para se ganhar o jogo. Note, portanto, que esta é uma implementação genérica, permitindo não apenas tabuleiros de 3x3. Em nossa implementação, o primeiro jogador utiliza X e o segundo utiliza O.

In [ ]:
# state representation as tuple, where 'to_move' denotes who moves next, 'utility' 
# denotes the utility of the current board state, 'board' denotes the state utility, 
# and 'moves' denotes which movements are possible from current state
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

# game class
class TicTacToe(Game):
    """Play TicTacToe on an h x v board, with Max (first player) playing 'X'.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions, and a board, in the form of
    a dict of {(x, y): Player} entries, where Player is 'X' or 'O'."""

    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """Legal moves are any square not yet taken."""
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        string = ''
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                string += ' ' + board.get((x, y), '.') + ' '
                if y != self.v:
                  string += '|'
            string += '\n'
            if x != self.h:
              string += '-'*(self.v*4-1) + '\n'
        output.clear()
        print(string, flush=True)
        time.sleep(1)

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'X' else -1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k

Para criar uma instância do jogo, basta fazer como segue.

In [ ]:
ttt = TicTacToe()

# print the initial state board
ttt.display(ttt.initial)

Agora podemos criar os jogadores. Nosso primeiro jogador é bem simples, jogando de forma totalmente aleatória independente do estado atual. Naturalmente, apenas ações possíveis são escolhidas. 

In [ ]:
def random_player(game, state):
    return random.choice(game.actions(state)) if game.actions(state) else None

Nosso segundo jogador utiliza o algoritmo Alpha-Beta Pruning. Para tal, o jogador recebe um estado e explora toda a árvore do jogo a partir dali (com exceção daquelas porções que são podadas). 

In [ ]:
def alpha_beta_player(game, state):
    return alpha_beta_search(state, game)

def alpha_beta_search(state, game):
    
    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

Finalmente, podemos iniciar o jogo. Para tal, basta executar a função `play_game` da nossa instância de `TicTacToe`. A função retorna a utilidade final sob a perspectiva do jogador 1. 

Neste primeiro teste, vamos colocar dois jogadores aleatórios para competir.

In [ ]:
result = ttt.play_game(random_player, random_player)
winner = {1: 'Player 1', 0: 'Draw', -1: 'Player 2'}
print('Result: %d' % result)
print('Winner: %s' % winner[result])

Note que, como os jogadores são aleatórios, o resultado do jogo varia de uma execução para outra. Para testar, rode o código acima mais de uma vez e compare os resultados.

Agora, vamos ver como o jogador aleatório se sai contra o jogador `alpha_beta_player`.

In [ ]:
result = ttt.play_game(alpha_beta_player, random_player)
winner = {1: 'Player 1', 0: 'Draw', -1: 'Player 2'}
print('Result: %d' % result)
print('Winner: %s' % winner[result])

Note que o jogador aleatório tem poucas chances contra o seu oponente. Na maioria das vezes, é o `alpha_beta_player` quem ganha. Para testarmos melhor, vamos repetir o jogo 20 vezes. Ao final, mostraremos o somatório dos resultados dos jogos. Se o somatório for menor que 20, então significa que o jogador aleatório ganhou pelo menos uma vez.

In [ ]:
repetitions = 20
summation = 0
for i in range(repetitions):
  summation += ttt.play_game(alpha_beta_player, random_player, animate=False)

print('Result: %d/%d' % (summation, repetitions))

Finalmente, vamos ver agora o que acontece quando dois jogadores do tipo `alpha_beta_player` jogam um contra o outro.

In [ ]:
result = ttt.play_game(alpha_beta_player, alpha_beta_player)
winner = {1: 'Player 1', 0: 'Draw', -1: 'Player 2'}
print('Result: %d' % result)
print('Winner: %s' % winner[result])

Neste cenário, note que o jogo sempre terminará em empate. Como ambos jogadores conhecem as melhores jogadas a cada estado, o empate se torna inevitável.

### Exercício 1

Calcule os valores Minimax de cada estado da árvore de jogo abaixo. 

![alt text](https://i.imgur.com/1mrTV5y.png)

Os valores Minimax dos estados são:
* Estado 'A': 4
* Estado 'B': -5
* Estado 'C': 4
* Estado 'D': -1
* Estado 'E': 0

### Exercício 2

Com base no algoritmo Alpha-Beta pruning, encontre os valores alpha e beta de cada estado da árvore de jogo abaixo. Lembre-se que este algoritmo poda porções da árvore; portanto, não deixe de indicar as porções podadas pelo algoritmo.

![alt text](https://i.imgur.com/zII14oY.png)

Os valores alpha-beta dos estados são:
* Estado 'A': [10, 10]
* Estado 'B': [10, 10]
* Estado 'C': [-inf, 8]
* Estado 'D': [25, 25]
* Estado 'E': [10, 10]
* Estado 'F': [8, 8]
* Estado 'G': [-inf, +inf]
* Estado 'H': [25, 25]
* Estado 'I': [-inf, -31]
* Estado 'J': [10, 10]
* Estado 'K': [-inf, -5]
* Estado 'L': [8, 8]
* Estado 'M': [-inf, 0]
* Estado 'N': [-inf, +inf]
* Estado 'O': [-inf, +inf]

As porções podadas da árvore são (identifiquei as arestas com base no nome das ações; cada aresta listada abaixo indica que a subárvore correspondente foi eliminada):
* Aresta 'i2'
* Aresta 'k2'
* Aresta 'm2'
* Aresta 'c2'


### Exercício 3

Crie uma classe para representar o jogo do Exercício 1. Em seguida, teste os algoritmos `alpha_beta_player` e `random_player` e comente os resultados.

In [ ]:
GameState = namedtuple('GameStateEx1', 'to_move, utility, node, moves')

class JogoEx1(Game):
    
    def __init__(self):
        self.moves = {'A': {'a1': 'B',   'a2': 'C',   'a3': 'D',   'a4': 'E'},
                      'B': {'b1': 'FB1', 'b2': 'FB2', 'b3': 'FB3', 'b4': 'FB4'},
                      'C': {'c1': 'FC1', 'c2': 'FC2', 'c3': 'FC3', 'c4': 'FC4'},
                      'D': {'d1': 'FD1', 'd2': 'FD2', 'd3': 'FD3', 'd4': 'FD4'},
                      'E': {'e1': 'FE1', 'e2': 'FE2', 'e3': 'FE3', 'e4': 'FE4'}}
                      
        self.utilities = {'FB1': -5, 'FB2': 8, 'FB3': 0,  'FB4': 3,
                          'FC1': 10, 'FC2': 8, 'FC3': 6,  'FC4': 4,
                          'FD1': 0,  'FD2': 2, 'FD3': -1, 'FD4': 5,
                          'FE1': 3,  'FE2': 2, 'FE3': 20, 'FE4': 0}
        
        self.initial = GameState(to_move='J1', utility=0, node='A', moves=list(self.moves['A'].keys()))

    # igual
    def actions(self, state):
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        
        new_node = self.moves[state.node][move]
        return GameState(to_move=('J2' if state.to_move == 'J1' else 'J1'),
                         utility=(self.utilities[new_node] if new_node in self.utilities else 0),
                         node=new_node, 
                         moves=(list(self.moves[new_node].keys()) if new_node in self.moves else None))

    def utility(self, state, player):
        return state.utility if player == 'J1' else -state.utility

    def terminal_test(self, state):
        return not state.node in self.moves

In [ ]:
jex1 = JogoEx1()
result = jex1.play_game(alpha_beta_player, alpha_beta_player)

print('Player 1: %d' % (result))
print('Player 2: %d' % (-result))

if result > 0:
  print('Player 1 wins!')
elif result < 0:
  print('Player 2 wins!')
else:
  print('Draw!')

### Exercício extra

Generalize a classe de jogo criada no Exercício 3 para que ela acomode jogos de múltiplos rounds, como o do Exercício 2. Em seguida, teste os algoritmos `alpha_beta_player` e `random_player` e comente os resultados.


In [ ]:
# TODO: your answer here